In [4]:
import pandas as pd
import re
from datetime import datetime
from dateutil.relativedelta import relativedelta

file_path = 'FIFA_DATASET_FINAL.csv'
df = pd.read_csv(file_path, low_memory=False, index_col=0)


In [5]:

'''
@name : weight()
@author : Suraj Nammi (snammi@hawk.iit.edu)
@purpose : To convert the weight values to a standardized format.
        Convert all values to kgs (LBs to KGs)
@param : value - value of a df column that needs to be converted
@return : updated value
'''
def weight(x):
    nu = int (re.findall('\d+', x )[0])
    strr = re.findall("[a-zA-Z]+", x)
    if strr[0] == 'lbs':
        result = nu * 0.453
        result = round(result)
        result = int(result)
    else:
        result = nu
    return result

df['Value'] = df['Value'].str.replace('€','') #removing garbage values
df['Wage'] = df['Wage'].str.replace('€','')  #removing garbage values
df['Release Clause'] = df['Release Clause'].str.replace('€','') #removing garbage values


'''
@name : cur()
@author : Suraj Nammi (snammi@hawk.iit.edu)
@purpose : To convert the currency values to a standardized format.
        Convert all values to thousands (M to K -> Millions to Thousand) Euros
@param : value - value of a df column that needs to be converted
@return : updated value
'''
def cur(x):
    whole = str (re.findall('\d+', x )[0])
    decimal = str (re.findall('\d+\.?(\d*)?', x )[0])
    nu = float(whole + "." + decimal) 
    
    strr = re.findall("[a-zA-Z]+", x)
    
    if len(strr) != 0 and 'M' == strr[0] :
        result = nu * 1000
        result = float(result)
    else:
        result = nu
    return result 


'''
@name : height()
@author : Shubham Singh (ssingh127@hawk.iit.edu)
@purpose : To convert the height values to a standardized format.
        Convert all values to cms (Feet and Inches to CMs)
@param : value - value of a df column that needs to be converted
@return : updated value
'''
def height(x):
    if "'" in x:
        feet = x.split("'")[0]
        inch = (x.split("'")[1]).split('"')[0]
        result = float(feet + "."  + inch) * 30.48
    else:   # when height is in cms
        result = float(re.findall('\d+', x )[0])
    return result


In [6]:
'''
@name : renameColumns()
@author : Purva Lila (plila1@hawk.iit.edu)
@purpose : To rename columns 
@param : dict - Dictionary containing Previous Column Name to New Column Name mapping in the dataframe object
                Example : {'ID': 'EmpID'}
'''
def renameColumns(dict):
    df.rename(columns=(dict), inplace=True)

    
'''
@name : getYearFrom()
@author : Shubham Singh (ssingh127@hawk.iit.edu)
@purpose : To retrieve an year value from date column in the dataframe object
           Expected Format of the Date column = 29-Jan-2020 —> (dd-mon-yyyy) —> ('%d-%b-%Y')
@param : columnName - Date column Name in the dataframe object
@param : index - index of the value in the entire dataframe for which year needs to be extracted
@return : year value
'''
def getYearFrom(columnName, index):
    date = df.iat[index, df.columns.get_loc(columnName)]
    date = datetime.strptime(date, '%d-%b-%Y').date()
    return date.year


'''
@name : valueAt()
@author : Suraj Nammi (snammi@hawk.iit.edu)
@purpose : To retrieve a specific value at an index from a column in the dataframe object
@param : columnName - column Name in the dataframe object
@param : index - index of the value in the entire dataframe for which value needs to be extracted
@return : value of the column at given index
'''
def valueAt(columnName, index):
    return df.iat[index, df.columns.get_loc(columnName)]


'''
@name : modifyDateFormat()
@author : Shubham Singh (ssingh127@hawk.iit.edu)
@purpose : To modify date format from '%d-%b-%y' to '%d-%b-%Y'
           Example : 29-Jan-20 will become 29-Jan-2020
@param : columnName - column Name in the dataframe object
@param : index - index of the value in the entire dataframe for which value needs to be extracted
'''
def modifyDateFormat(columnName, index):
    date = valueAt(columnName, index)
    date = datetime.strptime(date, '%d-%b-%y').date()
    date = date.strftime('%d-%b-%Y')
    df.iat[index, df.columns.get_loc(columnName)] = date

    
'''
@name : modifyContractLoanRelation()
@author : Purva Lila (plila1@hawk.iit.edu)
@purpose : Some columns such as (Contract, Joined, Loan Date End) were linked together and had variety of values.
           Due to the varied values there were some anomalies caused in the data.
           Bringing data to one standard format is done by modifyContractLoanRelation() method.

--------------------------------- Original Data --------------------------------- 
Contract	              Joined	       Loan Date End
2020 ~ 2025	              29-Jan-20	     NaN
Jun 30, 2023 On Loan	  9-Sep-20	     30-Jun-23
Free 	                  1-Jan-11

--------------------------------- Resultant Data --------------------------------- 
Contract          contractStartDate      contractEndDate     OnLoan
2020 ~ 2025       29-Jan-2020            29-Jan-2025         NO
2020 ~ 2023       9-Sep-2020             30-Jun-2023         YES
2011 ~ 9999 	  1-Jan-2011	         31-Dec-9999         FREE


--------------------------------- Operations Performed --------------------------------- 
# Contract —> startYear ~ endYear —> value calculation for "On Loan" entries 
# Joined —> renamed to contractStartDate —> No change in values
# Loan Date End —> renamed to contractEndDate  —> value calculation for without "On Loan" entries 
# onLoan —> New column added —> values = (YES / NO / FREE)

'''
def modifyContractLoanRelation():
    renameColumns({ 'Joined': 'contractStartDate'})
    renameColumns({ 'Loan Date End': 'contractEndDate'})
    df["OnLoan"] = pd.NaT    

    for index in range(df.shape[0]):
        x =  valueAt('Contract', index)

        modifyDateFormat('contractStartDate', index)

        if " ~ " in x:
            ''' Add number of years from contract in startDate to get endDate '''
            arr = x.split(" ~ ")
            joinDate = valueAt('contractStartDate', index)
            joinDate = datetime.strptime(joinDate, '%d-%b-%Y').date()
            endDate = joinDate +  relativedelta(years= int(arr[1]) - int(arr[0]))
            endDate = endDate.strftime('%d-%b-%Y')

            df.iat[index, df.columns.get_loc('contractEndDate')] = endDate
            df.iat[index, df.columns.get_loc('OnLoan')] = 'NO'
        
        elif 'On Loan' in x:
                ''' Extract years from both contractStartDate and contractEndDate to fill in Contract Field '''
                modifyDateFormat('contractEndDate', index)
                startYear = getYearFrom('contractStartDate', index)
                endYear = getYearFrom('contractEndDate', index)
                df.iat[index, df.columns.get_loc('Contract')] = str(startYear) + " ~ " + str(endYear)
                df.iat[index, df.columns.get_loc('OnLoan')] = 'YES'

        elif 'Free' in x: 
            endDate = datetime.strptime('31-Dec-9999', '%d-%b-%Y').date()
            endDate = endDate.strftime('%d-%b-%Y')
            df.iat[index, df.columns.get_loc('contractEndDate')] = endDate
            df.iat[index, df.columns.get_loc('OnLoan')] = 'FREE'

            ''' Extract years from both contractStartDate and contractEndDate to fill in Contract Field '''
            startYear = getYearFrom('contractStartDate', index)
            endYear = getYearFrom('contractEndDate', index)
            df.iat[index, df.columns.get_loc('Contract')] = str(startYear) + " ~ " + str(endYear)       

        else:
            ''' If both kinds of expected values doesn't exists, just assign NaN '''
            df.iat[index, df.columns.get_loc('Contract')] = pd.NaT
            df.iat[index, df.columns.get_loc('contractStartDate')] = pd.NaT
            df.iat[index, df.columns.get_loc('contractEndDate')] = pd.NaT
            df.iat[index, df.columns.get_loc('OnLoan')] = pd.NaT


In [7]:
df['Weight'] = df['Weight'].apply(weight)
print('Weight Column Data is Cleaned')   

df['Value'] = df['Value'].apply(cur)
print('Values Column Data is Cleaned')

df['Wage'] = df['Wage']
print('Wage Column Data is Cleaned')

df['Release Clause'] = df['Release Clause'].apply(cur)
print('Release Clause Data is Cleaned')

df['Height'] = df['Height'].apply(height)
print('Height Column Data is Cleaned')

renameColumns({ 'Value': 'Value_Thousand_Euros'})
renameColumns({ 'Wage': 'Wage_Thousand_Euros'})
renameColumns({ 'Release_Clause': 'Release_Clause_Thousand_Euros'})
renameColumns({ 'Weight': 'Weight_KG'})
renameColumns({ 'Height': 'Height_cm'})

renameColumns({ '↓OVA': 'Over All Rating'})
print('↓OVA Column is renamed to Over All Rating')

modifyContractLoanRelation()
print('Contract Loan Relationship is Cleaned')

df.to_csv('Cleaned Dataset.csv')
print('CSV is updated to Cleaned Dataset.csv')


Weight Column Data is Cleaned
Values Column Data is Cleaned
Wage Column Data is Cleaned
Release Clause Data is Cleaned
Height Column Data is Cleaned
↓OVA Column is renamed to Over All Rating
Contract Loan Relationship is Cleaned
CSV is updated to Cleaned Dataset.csv
